# Import data

In [3]:
import gdown
import zipfile

import pandas as pd 
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
import pickle

import re
import string
import emoji

import spacy
import nltk
from nltk.tokenize import TweetTokenizer
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)


2021-07-11 19:15:22,146 [16099] WARNING  py.warnings: /home/user/Data-science/venv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)



## Download files from Google Drive

In [4]:
url = 'https://drive.google.com/uc?id=1jI1cmxqnwsmC-vbl8dNY6b4aNBtBbKy3'
output = 'Twitter.zip'
gdown.download(url, output, quiet=False) 

Downloading...
From: https://drive.google.com/uc?id=1jI1cmxqnwsmC-vbl8dNY6b4aNBtBbKy3
To: /home/user/Data-science/visualization/scattertext/Twitter.zip
120MB [00:10, 11.8MB/s] 


'Twitter.zip'

In [6]:
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('.')

## Read Data

In [2]:
path_train = 'Data/train/en'
path_test = 'Data/test/en'

In [3]:
tweets_train = [f for f in listdir(path_train) if isfile(join(path_train, f))]
tweets_test = [f for f in listdir(path_test) if isfile(join(path_test, f))]

In [ ]:
for r in ET.parse(join(path_train,tweets_train[6])).getroot()[0]:
    print(r.text)

In [6]:
#ET.parse converts the file into the tree
def convert_texts(path_train, files_train):
    
    doc = []
    for i in range(len(files_train)):
        #Append the tweets to the corresponding document
        try:
            doc1 =[r.text for r in ET.parse(join(path_train,files_train[i])).getroot()[0]]
            doc.append(' '.join(t for t in doc1))
        except:
            pass 
    return doc 

In [12]:
# list of train and test tweets
t_train = convert_texts(path_train, tweets_train)
t_test  = convert_texts(path_test, tweets_test)

# Get Labels

In [13]:
def get_labels(path, files):
    dic={}
    task=[[],[]]
    for e in open(join(path,'truth.txt')).read().split('\n'):
        d=e.split(':::')
        if(len(d)==3):
            dic.update({d[0]: d[1:]})
            task[0].append(d[1])
            task[1].append(d[2])
    task=[sorted(list(set(t))) for t in task]
    
    d_task = [{item:i for i, item in enumerate(t)} for t in task]
    labels=[[],[]]

    for e in files:
        try:
            ID=e[:-4]
            labels[0].append(d_task[0][dic[ID][0]])
            labels[1].append(d_task[1][dic[ID][1]])
        except:
            pass
    print(d_task)
    return np.array(labels)

In [17]:
labels_train = get_labels(path_train, tweets_train)
labels_test  = get_labels(path_test , tweets_test)
labels_train.shape

[{'female': 0, 'male': 1}, {'australia': 0, 'canada': 1, 'great britain': 2, 'ireland': 3, 'new zealand': 4, 'united states': 5}]
[{'female': 0, 'male': 1}, {'australia': 0, 'canada': 1, 'great britain': 2, 'ireland': 3, 'new zealand': 4, 'united states': 5}]


(2, 3600)

In [18]:
GENDER_MAP = {0:'female', 1: 'male'}
COUNTRY_MAP = {0: 'australia', 1: 'canada', 2: 'great britain', 3: 'ireland', 4: 'new zealand', 5: 'united states'}

# Preprocessing

In [20]:
replacement_patterns = [
    #match url (i.e: https://t.co/5tF5G9VKtq)
    (r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ''),

    #match user (i.e: @cerpintor )
    (r'@\w+', ''),

    #match hashtag (i.e: #WomensMarchOnWashington)
    (r'#\w+', ''),

    #Replace "&..." with ''
    (r'&\w+', '')
]

class RegexReplacer(object):
    def __init__(self, patterns = replacement_patterns):
        self.patterns = [(re.compile(regrex),repl) for (regrex, repl) in
                        patterns]
    
    #Replace the words that match the patterns with replacement words
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        return s

In [21]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [22]:
def lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    
    doc = nlp(" ".join(text)) 
    return [token.lemma_ for token in doc if token.pos_ in allowed_postags]


In [23]:
nlp = spacy.load('en', disable=['parser', 'ner'])
tknz = TweetTokenizer()
replacer = RegexReplacer()

# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']
# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)
# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)

# punctuation
punc = string.punctuation

In [24]:
def normalize(doc):
    
    for i in range(len(doc)):
        
        doc[i] = give_emoji_free_text(doc[i])
        
        #Tokenize with replacement
        doc[i] = tknz.tokenize(replacer.replace(doc[i]))
        
        #Filter stopwords, punctuations, and lowercase
        doc[i] = [w.lower() for w in doc[i] if w not in punc and w not in ALL_STOP_WORDS]
        
        # Lemmatize
        doc[i] = lemmatization(doc[i], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    return doc

In [25]:
t_train_processed = normalize(t_train)

In [ ]:
t_train_processed[0]

In [27]:
t_train_processed = np.array(t_train_processed)
t_train_processed = np.expand_dims(t_train_processed, 1)

combine_arr = np.concatenate((t_train_processed, labels_train.T), axis=1)

In [32]:
df = pd.DataFrame(combine_arr, columns=['tweets', 'gender', 'country'])

In [34]:
df.gender = df.gender.map(GENDER_MAP)
df.country = df.country.map(COUNTRY_MAP)

In [35]:
df 

,tweets,gender,country
0,"[fucking, terrify, nemesis, record, bad, enemy...",male,canada
1,"[poetry, shelf, summer, season, poet, pick, po...",female,new zealand
2,"[say, truth, attest, cloud, sky, joke, how, ok...",female,canada
3,"[seem, right, pretty, accurate, new, breakfast...",female,canada
4,"[s, word, touch, early, leave, night, morning,...",male,ireland
...,...,...,...
3595,"[love, xma, beautiful, spot, swim, fancy, glas...",female,australia
3596,"[late, genomic, news, thank, late, genomic, ne...",male,australia
3597,"[think, well, year, early, time, tune, et, cat...",female,united states
3598,"[find, cojone, sort, keep, diggin, racist, rea...",female,united states


In [36]:
df.to_pickle('Data/processed_train_data.pkl')